# Compare EPViT with the baselines CLIP and histogram

In [1]:
import relational_image_generation_evaluation as rige

In [7]:
dataset_daa = rige.get_adversarial_attribute_dataset()
def get_prompt(sample):
    sid, oid = list(sample.edges)[0]
    rel = sample.edges[(sid, oid)]['predicate']
    os = sample.nodes[sid]['name']
    ats = sample.nodes[sid]['attributes']
    oo = sample.nodes[oid]['name']
    ato = sample.nodes[oid]['attributes']
    prompt = f"{','.join(ats)} {os} {rel} {','.join(ato)} {oo}".strip().lower()
    prompt += '.'
    return prompt
for sample in dataset_daa:
    print(f"Image ID: {sample['original_graph'].image_id}")
    print(f"Original Prompt: {get_prompt(sample['original_graph'])}")
    print(f"Adversarial Prompt: {get_prompt(sample['adv_graph'])}")
    print()


Image ID: 107907
Original Prompt: None
Adversarial Prompt: None

Image ID: 107907
Original Prompt: None
Adversarial Prompt: None

Image ID: 107910
Original Prompt: None
Adversarial Prompt: None

Image ID: 107910
Original Prompt: None
Adversarial Prompt: None

Image ID: 107910
Original Prompt: None
Adversarial Prompt: None

Image ID: 107910
Original Prompt: None
Adversarial Prompt: None

Image ID: 107910
Original Prompt: None
Adversarial Prompt: None

Image ID: 107915
Original Prompt: None
Adversarial Prompt: None

Image ID: 107947
Original Prompt: None
Adversarial Prompt: None

Image ID: 150476
Original Prompt: None
Adversarial Prompt: None

Image ID: 150476
Original Prompt: None
Adversarial Prompt: None

Image ID: 150502
Original Prompt: None
Adversarial Prompt: None

Image ID: 150502
Original Prompt: None
Adversarial Prompt: None

Image ID: 150502
Original Prompt: None
Adversarial Prompt: None

Image ID: 150502
Original Prompt: None
Adversarial Prompt: None

Image ID: 713540
Original

### Datasets

In [4]:
help(dataset_daa)

Help on list object:

class list(object)
 |  list(iterable=(), /)
 |  
 |  Built-in mutable sequence.
 |  
 |  If no argument is given, the constructor creates a new empty list.
 |  The argument must be an iterable if specified.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, value, /)
 |      Return self+value.
 |  
 |  __contains__(self, key, /)
 |      Return key in self.
 |  
 |  __delitem__(self, key, /)
 |      Delete self[key].
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __getitem__(...)
 |      x.__getitem__(y) <==> x[y]
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iadd__(self, value, /)
 |      Implement self+=value.
 |  
 |  __imul__(self, value, /)
 |      Implement self*=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate sign

In [2]:
dataset_daa = rige.get_adversarial_attribute_dataset()

def graph_to_caption(graph):
    sid, oid = list(graph.edges)[0]
    rel = graph.edges[(sid, oid)]['predicate']
    os = graph.nodes[sid]['name']
    ats = graph.nodes[sid]['attributes']
    oo = graph.nodes[oid]['name']
    ato = graph.nodes[oid]['attributes']
    rel_str = f"{','.join(ats)} {os} {rel} {','.join(ato)} {oo}".strip().lower()
    rel_str += '.'
    return rel_str

from copy import deepcopy
import random

random.seed(123456)

dataset_oneedge = rige.OneEdgeDataset(testonly=False)
dataset_oneedge = list(dataset_oneedge)

random.shuffle(dataset_oneedge)
dataset_eaa = []
for graph in dataset_oneedge:
    assert len(list(graph.nodes)) == 2
    id1, id2 = list(graph.nodes)
    if len(graph.nodes[id1]['attributes']) != 1 or len(graph.nodes[id2]['attributes']) != 1:
        continue
    graph.caption = graph_to_caption(graph)
    adv_graph = deepcopy(graph)
    adv_graph.nodes[id1]['attributes'], adv_graph.nodes[id2]['attributes'] = \
        adv_graph.nodes[id2]['attributes'], adv_graph.nodes[id1]['attributes']
    adv_graph.caption = graph_to_caption(adv_graph)
    d = {
        'original_graph': graph,
        'adv_graph': adv_graph,
    }
    dataset_eaa.append(d)
    if len(dataset_eaa) == 100:
        break
    
dataset_dar = rige.get_adversarial_relationship_dataset()
dataset_darv2 = rige.get_adversarial_relationship_dataset(version='v2')


dataset_ear = []
for graph in dataset_oneedge:
    assert len(list(graph.nodes)) == 2
    id1, id2 = list(graph.nodes)
    if len(graph.nodes[id1]['attributes']) != 0 or len(graph.nodes[id2]['attributes']) != 0:
        continue
    graph.caption = graph_to_caption(graph)
    adv_graph = deepcopy(graph)
    adv_graph.nodes[id1]['name'], adv_graph.nodes[id2]['name'] = \
        adv_graph.nodes[id2]['name'], adv_graph.nodes[id1]['name'] 
    adv_graph.caption = graph_to_caption(adv_graph)
    d = {
        'original_graph': graph,
        'adv_graph': adv_graph,
    }
    dataset_ear.append(d)
    if len(dataset_ear) == 100:
        break

datasets = [("daa", dataset_daa), ("eaa", dataset_eaa), ("dar", dataset_dar), ("ear", dataset_ear), ("dar", dataset_darv2)]
# datasets = [("darv2", dataset_darv2)]

Loading filtered graphs...
Finished loading filtered graphs
Generating one edge graphs...


100%|██████████| 97216/97216 [01:50<00:00, 876.33it/s] 


Finished generating one edge graphs


In [3]:
dataset_darv2 = rige.get_adversarial_relationship_dataset(version='v2')
datasets = [("darv2", dataset_darv2)]

In [4]:
datasets = [("daa", dataset_daa), ("eaa", dataset_eaa), ("dar", dataset_dar), ("ear", dataset_ear), ("dar", dataset_darv2)]


In [5]:
image_ids = []
for _, d in datasets:
    for g in d:
        image_ids.append(g['original_graph'].image_id)
        image_ids.append(g['adv_graph'].image_id)

In [6]:
image_ids = list(set(image_ids))
image_ids.sort()
len(image_ids)

367

In [7]:
image_ids

[2,
 4,
 6,
 10,
 11,
 17,
 18,
 21,
 44,
 45,
 54,
 73,
 76,
 81,
 84,
 85,
 87,
 89,
 102,
 106,
 115,
 166,
 174,
 191,
 213,
 239,
 260,
 271,
 329,
 334,
 366,
 375,
 399,
 418,
 528,
 531,
 577,
 601,
 910,
 1945,
 2267,
 2566,
 2831,
 3144,
 3586,
 3648,
 3679,
 3732,
 4435,
 4702,
 107907,
 107910,
 107915,
 107947,
 150476,
 150502,
 285981,
 286028,
 713367,
 713381,
 713502,
 713540,
 1159360,
 1159875,
 1159877,
 1159922,
 1159940,
 1160193,
 1592360,
 1592645,
 1593095,
 2315536,
 2316858,
 2316927,
 2317402,
 2317571,
 2317714,
 2317738,
 2317779,
 2318531,
 2318546,
 2319238,
 2319263,
 2319559,
 2319593,
 2319713,
 2320161,
 2320262,
 2320680,
 2321244,
 2321504,
 2321841,
 2321904,
 2322239,
 2322246,
 2322690,
 2323437,
 2323585,
 2324105,
 2324290,
 2324401,
 2324544,
 2324666,
 2324928,
 2324995,
 2325181,
 2325483,
 2325754,
 2325969,
 2327126,
 2327561,
 2327844,
 2328306,
 2328437,
 2328995,
 2329020,
 2329089,
 2329609,
 2329926,
 2329988,
 2330300,
 2330389,
 2

In [19]:
for i in rnage()
g = dataset_dar[0]['original_graph']
g.nodes[list(g.nodes)[0]]

{'w': 28, 'h': 97, 'x': 6, 'y': 226, 'attributes': [], 'name': 'man'}

In [20]:
for i in range(100):
    print(dataset_dar[i]['adv_graph'].caption, )
    print(dataset_dar[i]['original_graph'].caption)
    g = dataset_dar[i]['original_graph']
    print(g.nodes[list(g.nodes)[0]])
    print()

man lying on sidewalk.
man walking on sidewalk.
{'w': 28, 'h': 97, 'x': 6, 'y': 226, 'attributes': [], 'name': 'man'}

man sitting on sidewalk.
man walking on sidewalk.
{'w': 28, 'h': 97, 'x': 6, 'y': 226, 'attributes': [], 'name': 'man'}

book beside table.
book on table.
{'w': 452, 'h': 62, 'x': 100, 'y': 218, 'attributes': [], 'name': 'table'}

man beside sidewalk.
man on sidewalk.
{'w': 78, 'h': 217, 'x': 223, 'y': 305, 'attributes': [], 'name': 'man'}

man behind sidewalk.
man on sidewalk.
{'w': 78, 'h': 217, 'x': 223, 'y': 305, 'attributes': [], 'name': 'man'}

man lying on desk.
man sitting at desk.
{'w': 128, 'h': 149, 'x': 576, 'y': 270, 'attributes': [], 'name': 'man'}

cup next to desk.
cup on desk.
{'w': 472, 'h': 231, 'x': 0, 'y': 368, 'attributes': [], 'name': 'desk'}

cup in front of desk.
cup on desk.
{'w': 472, 'h': 231, 'x': 0, 'y': 368, 'attributes': [], 'name': 'desk'}

bottle beside table.
bottle on table.
{'w': 49, 'h': 93, 'x': 210, 'y': 251, 'attributes': [], 'n

### Evaluation

In [15]:
from PIL import Image
def evaluate(evaluator, dataset):
    orig_graphs = []
    adv_graphs = []
    images = []
    for sample in dataset:
        orig_graph = sample['original_graph']
        orig_graphs.append(orig_graph)
        adv_graph = sample['adv_graph']
        adv_graphs.append(adv_graph)
        image_id = orig_graph.image_id
        assert orig_graph.image_id == adv_graph.image_id
        # adapt to your local directory
        try:
            IMAGE_DIR = '../datasets/visual_genome/raw/VG_100K/'
            image = Image.open(IMAGE_DIR + str(image_id) + '.jpg')
        except:
            IMAGE_DIR = '../datasets/visual_genome/raw/VG_100K_2/'
            image = Image.open(IMAGE_DIR + str(image_id) + '.jpg')
        images.append(image)
    orig_scores = evaluator(images, orig_graphs)
    print(".............................")
    adv_scores = evaluator(images, adv_graphs)
    
    if 'rel_scores' in orig_scores:
    # EPViT...
        n_correct = 0
        n_total = len(dataset)
        n_equal = 0
        for i in range(n_total):
            # print(orig_scores['rel_scores'][i], orig_scores['attr_scores'][i])
            if orig_scores['attr_scores'][i] == 'noattributes':
                orig_score = orig_scores['rel_scores'][i]
            else:
                orig_score = orig_scores['rel_scores'][i] + orig_scores['attr_scores'][i]
            if adv_scores['attr_scores'][i] == 'noattributes':
                adv_score = adv_scores['rel_scores'][i]
            else:
                adv_score = adv_scores['rel_scores'][i] + adv_scores['attr_scores'][i]
            if orig_score > adv_score:
                n_correct += 1
            elif orig_score == adv_score:
                n_correct += 0.5
                n_equal += 1
        print("n_equal", n_equal)
        acc = n_correct / n_total
    elif 'overall_scores' in orig_scores:
        n_correct = 0
        n_total = len(dataset)
        for i in range(n_total):
            orig_score = orig_scores['overall_scores'][i]
            adv_score = adv_scores['overall_scores'][i]
            if orig_score > adv_score:
                n_correct += 1
            elif orig_score == adv_score:
                n_correct += 0.5
        acc = n_correct / n_total
    res = {
        'acc': acc
    }
    return res

#### EPViT

In [27]:
evaluator_epvit = rige.Evaluator('ViT-L/14')
for name, dataset in datasets:
    acc = evaluate(evaluator_epvit, dataset)
    print('epvit', name, acc)
del evaluator_epvit

Using device cuda:4 for evaluation.
Using text embeddings as input to the model.
.............................
n_equal 0
epvit darv2 {'acc': 0.7936507936507936}


#### CLIP ViT-L/14

In [28]:
evaluator_clip = rige.Evaluator('CLIP_ViT-L/14')
for name, dataset in datasets:
    acc = evaluate(evaluator_clip, dataset)
    print('clip-l/14', name, acc)
del evaluator_clip

Using device cuda:5 for evaluation.
Downloaded weights for CLIP_ViT-L/14 to /local/home/rhoenig/GraphCLIP/relational_image_generation_evaluation/relational_image_generation_evaluation/data/dummy.ckpt.
.............................
clip-l/14 darv2 {'acc': 0.5714285714285714}


#### CLIP ViT-G/14

In [38]:
evaluator_clip = rige.Evaluator('CLIP_ViT-G/14')
for name, dataset in [("dar", dataset_dar)]:
    acc = evaluate(evaluator_clip, dataset)
    print('clip-G/14', name, acc)
del evaluator_clip

Using device cuda:3 for evaluation.
Downloaded weights for CLIP_ViT-G/14 to /local/home/rhoenig/GraphCLIP/relational_image_generation_evaluation/relational_image_generation_evaluation/data/dummy.ckpt.
.............................
clip-G/14 dar {'acc': 0.5631067961165048}


In [55]:
orig_captions = []
orig_captions_v2 = []
for i in range(len(dataset_dar)):
    orig_captions.append(dataset_dar[i]['original_graph'].caption)
    orig_captions_v2.append(dataset_darv2[i]['original_graph'].caption)

    # print(dataset_dar[i]['original_graph'].caption)
    # print(dataset_dar[i]['adv_graph'].caption)
    print()


IndexError: list index out of range

In [56]:
orig_captions.sort()
orig_captions_v2.sort()

In [57]:
orig_captions

['bike next to car.',
 'bike next to car.',
 'bird on branch.',
 'book on shelf.',
 'book on shelf.',
 'book on shelf.',
 'book on table.',
 'bottle on table.',
 'bottle on table.',
 'bottle on table.',
 'bottle on table.',
 'bowl on table.',
 'box on top of desk.',
 'box under desk.',
 'box under desk.',
 'building next to tower.',
 'building next to tower.',
 'car on road.',
 'cup on desk.',
 'cup on desk.',
 'cup on desk.',
 'door on cabinet.',
 'flowers in vase.',
 'flowers on table.',
 'food on table.',
 'handle on cup.',
 'hat on head.',
 'hill behind building.',
 'laptop on desk.',
 'laptop on desk.',
 'leaves on sign.',
 'man on bed.',
 'man on bed.',
 'man on sidewalk.',
 'man on sidewalk.',
 'man sitting at desk.',
 'man sitting at table.',
 'man walking on sidewalk.',
 'man walking on sidewalk.',
 'people at table.',
 'people in a boat.',
 'people sitting on top of building.',
 'people sitting on top of building.',
 'person sitting in car.',
 'person sitting on elephant.',
 

In [58]:
orig_captions_v2

['bear sitting on grass.',
 'bear sitting on grass.',
 'bear under umbrella.',
 'bear walking on grass.',
 'boy on bench.',
 'boy on couch.',
 'boy sitting in bed.',
 'boy sitting on bed.',
 'boy sitting on bench.',
 'building behind pole.',
 'building next to pole.',
 'bush beside clock.',
 'bush in front of clock.',
 'cat in bag.',
 'cat on bag.',
 'cat on top of bench.',
 'cat sitting on bench.',
 'chair in front of ocean.',
 'clock on building.',
 'clock outside building.',
 'dog beside bike.',
 'dog in a bed.',
 'dog in front of bike.',
 'dog lying on bed.',
 'dog near motorcycle.',
 'door next to toilet.',
 'fence next to people.',
 'girl next to bear.',
 'grass above bench.',
 'grass behind bus.',
 'grass in front of bench.',
 'grass in front of bus.',
 'house behind field.',
 'lady sitting on bench.',
 'leaves are on table.',
 'leaves on table.',
 'line in sand.',
 'line on sand.',
 'man laying on seat.',
 'man lying on bench.',
 'man sitting on bench.',
 'man sitting on seat.'

In [33]:
dataset_dar[0]['original_graph'].caption
dataset_dar[0]['adv_graph'].caption

'man lying on sidewalk.'

#### Histogram

In [16]:
evaluator_hist = rige.Evaluator('histogram')

for name, dataset in datasets:
    acc = evaluate(evaluator_hist, dataset)
    print('histogram', name, acc)
del evaluator_hist

Using device cuda:0 for evaluation.
.............................
histogram daa {'acc': 0.64}
.............................
histogram eaa {'acc': 0.85}
.............................
histogram dar {'acc': 0.8883495145631068}
.............................
histogram ear {'acc': 0.985}
.............................
histogram dar {'acc': 0.6825396825396826}


In [ ]:
evaluator_hist = rige.Evaluator('histogram')

for name, dataset in datasets:
    acc = evaluate(evaluator_hist, dataset)
    print('histogram', name, acc)
del evaluator_hist

Using device cuda:0 for evaluation.
.............................
histogram daa {'acc': 0.64}
.............................
histogram eaa {'acc': 0.78}
.............................
histogram dar {'acc': 0.8543689320388349}
.............................
histogram ear {'acc': 0.98}
.............................
histogram dar {'acc': 0.6349206349206349}


In [23]:
len(rige.get_adversarial_relationship_dataset(version='v1'))

103